# Hyperparameter Finetuning

We want to find the right parameters for the Generator Network.

By using the best values for the following parameters:

- Number of Epochs (meaning `num_epochs` and `num_steps`)
- Learning Rate
- Batch Size
- Number of Noise Batches
- Number of Layers
- Regularization term
- Number of Neurons for each Network

In [5]:
from src.fyemu_tunable import main
import torch
import os
import torchvision.transforms as tt
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import optuna
from torchvision.models import resnet18

from src.metrics import kl_divergence_between_models

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

n0 = 5000
n2 = 5000
batch_size = 4
data_dir = f'data{os.sep}cifar10'

transform_test = tt.Compose([
    tt.ToTensor(),
    tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

valid_ds = ImageFolder(data_dir+f'{os.sep}test', transform_test)
valid_dl = DataLoader(valid_ds, batch_size*2,)

In [ ]:

def objective(trial):

    global valid_dl

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Steps = trial.suggest_int('opt_Steps', 1, 10)
    opt_Learning_Rate = trial.suggest('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest('opt_Batch_Size', 1, 64)
    opt_Number_of_Noise_Batches = trial.suggest_int('opt_Number_of_Noise_Batches', 1, 10)
    opt_Regularization_term = trial.suggest('opt_Regularization_term', 0.01, 0.3)
    opt_Layers = trial.suggest_int('opt_Layers', 1, 10)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 1000)

    mod, histo = main(
        t_Epochs = opt_Epochs,
        t_Steps= opt_Steps,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_Number_of_Noise_Batches = opt_Number_of_Noise_Batches,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = [opt_Layers],
        t_Noise_Dim = opt_Noise_Dim,
        new_baseline=True,
        logs=True,
    )

    exact = resnet18(num_classes = 10).to(DEVICE)
    exact.load_state_dict(torch.load("ResNET18_CIFAR10_ALL_CLASSES.pt", weights_only=True))
    div = kl_divergence_between_models(
        model1 = mod,
        model2 = exact,
        data_loader = valid_dl,
    )

    return div

In [2]:
standard_model, standard_history = main(
    t_Epochs = 5,
    t_Steps= int((n0 + n2)/(2 * batch_size)), # The Idea is to have the same amount of updates as their are samples to unlearn
    t_Learning_Rate = 0.1,
    t_Batch_Size = batch_size,
    t_Number_of_Noise_Batches = 10,
    t_Regularization_term = 0.1,
    t_Layers = [1000],
    t_Noise_Dim = 100,
    new_baseline=True,
    logs=True,
)

['test', 'train']
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


c:\Users\Moritz\miniconda3\envs\bach.conda\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


---Training new ResNet18---


Training...:  27%|██▋       | 3406/12500 [07:36<20:19,  7.46batch/s]


KeyboardInterrupt: 

In [16]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt

data_dir = 'data\\cifar10'

In [17]:
transform_train = tt.Compose([
    tt.ToTensor(),
    tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = tt.Compose([
    tt.ToTensor(),
    tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_ds = ImageFolder(data_dir + '\\train', transform_train)
valid_ds = ImageFolder(data_dir + '\\test', transform_test)

batch_size = 8 # Changed
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size*2, num_workers=3, pin_memory=True)

In [20]:
classes_to_forget = [0, 1]

rt_tr = {}
for t, l in train_ds.imgs:
    if l not in classes_to_forget:
        rt_tr[len(rt_tr)] = (t, l)
rt_vl = {}
for t, l in valid_ds.imgs:
    if l not in classes_to_forget:
        rt_vl[len(rt_vl)] = (t, l)

In [21]:
from torch.utils.data import Dataset
from PIL import Image

class SubData(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        path, label = self.data[idx]
        img = Image.open(path).convert('RGB')
        img = self.transform(img)
        return img, label
    
rt_tr = SubData(rt_tr, transform_train)
rt_vl = SubData(rt_vl, transform_test)

rt_tr_dl = DataLoader(rt_tr, batch_size, shuffle=True)
rt_vl_dl = DataLoader(rt_vl, batch_size*2)

In [34]:
for i in rt_tr_dl:
    print(i[0].shape)
    print(i[1].shape)
    break

torch.Size([8, 3, 32, 32])
torch.Size([8])
